In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
import xml.etree.ElementTree as ET
import numpy as np
from matplotlib import pyplot
import re
from pathlib import Path
import pycountry
%matplotlib inline

# Introduction
We would like to get a rough estimate of the number of research participants for whom brain PET data has been collected yearly for the previous 5 to 10 years, both in the United States and abroad. Getting an estimate of the number of scans collected in the US is fairly straightforward, assuming that the ClinicalTrials.gov data is at least roughly accurate. The tricky part is coming up with an estimate for the number subjects collected worldwide. For that, we use a PubMed search to find all studies related to brain PET and get a ratio of US brain PET studies to non-US brain PET studies and estimate the number of worldwide subjects based on the this ratio in combination with the number of US subjects.

# Method
## Prepare data from clinicaltrials.gov

We searched on clinicaltrials.gov for "BRAIN" and "PET" in the other terms field on August 23rd, 2018 and found 1048 results (available in `https://raw.githubusercontent.com/nih-fmrif/PET_count/master/SearchResults.csv`). First we'll load in the search results, pull out the start year and completion year, then create an indicator variable for studies funded by the NIH or another US Federal source. 

In [2]:
ct_df = pd.read_csv('https://raw.githubusercontent.com/nih-fmrif/PET_count/master/SearchResults.csv')
# Check the data shape
assert ct_df.shape == (1048,27)
ct_df['start_year'] = ct_df['Start Date'].str[-4:].astype(float)
ct_df['complete_year'] = ct_df['Primary Completion Date'].str[-4:].astype(float)
ct_df['NIH_funding'] = ct_df['Funded Bys'].str.contains('NIH') | ct_df['Funded Bys'].str.contains('U.S. Fed')

## Prepare data from PubMed
We searched on https://www.ncbi.nlm.nih.gov/pubmed for "BRAIN" and  "PET" on August 23rd, 2018 and downloaded the results as XML (available zipped, here: https://github.com/nih-fmrif/PET_count/raw/master/pubmed_result.xml.zip). This search returned 22,770 results. We'll download these results, unzip them, then load the data into a dataframe

In [3]:
xml_path = Path('pubmed_result.xml')


# download the results and unzip them
if not xml_path.exists():
    ! wget https://github.com/nih-fmrif/PET_count/raw/master/pubmed_result.xml.zip
    ! unzip pubmed_result.xml.zip
    
xml_data = xml_path.read_text()
root = ET.XML(xml_data)

To load them into a dataframe we're going through all of the article objects and pulling out the article year and other date information, the PMID, Language, list of publication types, and infomration about the first and last author's affiliations. We'll need the affiliation data to guess which country the data was collected in.

In [4]:
pm_lod = []
for result in root.getchildren():
    dd = {}
    try:
        mlc = result.find('MedlineCitation')
        dd['PMID'] = mlc.find('PMID').text
        article = mlc.find('Article')
    except AttributeError:
        pm_lod.append(dd)
        continue
    
    dd['ArticleTitle'] = article.find('ArticleTitle').text
    dd['ArticleYear'] = np.nan
    dd['ArticleMonth'] = np.nan
    dd['ArticleDay'] = np.nan
    try:
        try:
            ArticleDate = article.find('ArticleDate')
            dd['ArticleYear'] = ArticleDate.find('Year').text
            dd['ArticleMonth'] = ArticleDate.find('Month').text
            dd['ArticleDay'] = ArticleDate.find('Day').text
        except AttributeError:
            try:
                ArticleDate = article.find('Journal').find('JournalIssue').find('PubDate')
                dd['ArticleYear'] = ArticleDate.find('Year').text
                dd['ArticleMonth'] = ArticleDate.find('Month').text
                dd['ArticleDay'] = ArticleDate.find('Day').text

            except AttributeError:
                dd['ArticleYear'] = np.ceil(np.mean([int(year) for year in re.findall('[0-9][0-9][0-9][0-9]',ArticleDate.find('MedlineDate').text)]))
    except AttributeError:
        pass
    dd['ArticleYear'] = float(dd['ArticleYear'])
    dd['PublicationTypeList'] = [pt.text for pt in article.find('PublicationTypeList').findall('PublicationType')]
    dd['Language'] = article.find('Language').text
    try:
        authlist = article.find('AuthorList').findall('Author')
    except AttributeError:
        pm_lod.append(dd)
        continue
    try:
        dd['FirstAuth_aff'] = '|'.join([affinfo.text for affinfo in authlist[0].find('AffiliationInfo').findall('Affiliation')])
    except AttributeError:
        dd['FirstAuth_aff'] = np.nan
    try:
        dd['LastAuth_aff'] = '|'.join([affinfo.text for affinfo in authlist[-1].find('AffiliationInfo').findall('Affiliation')])
    except AttributeError:
        dd['LastAuth_aff'] = np.nan
    pm_lod.append(dd)
assert(len(pm_lod) == len(root.getchildren()))
assert(len(pm_lod) == 22770)
       
pmdf = pd.DataFrame(pm_lod)

In [5]:
print("Number of records missing a last author affiliation: %d" %pd.isnull(pmdf.LastAuth_aff).sum())
print("Number of records missing a first author affiliation: %d" %pd.isnull(pmdf.FirstAuth_aff).sum())
print("Number of records missing both author affiliations: %d" %(pd.isnull(pmdf.LastAuth_aff) & pd.isnull(pmdf.FirstAuth_aff)).sum())

Number of records missing a last author affiliation: 16532
Number of records missing a first author affiliation: 1205
Number of records missing both author affiliations: 1150


We'll want to be able to remove reviews, as these are generally not going to be contributing new scans, so we create an indicator variable for publication types that are not reviews.

In [6]:
pmdf['not_review'] = False
pmdf.loc[pd.notnull(pmdf.PublicationTypeList),"not_review"] = pmdf.loc[pd.notnull(pmdf.PublicationTypeList),"PublicationTypeList"].apply(lambda x: 'Review' not in x)

In order to get affiliation, we'll need a list of country names from pycountry, as well as a list of state abbreviations and state names.

In [7]:

country_names = [x.name.lower() for x in pycountry.countries]    
statecodes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
statenames= [
    'District of Columbia',
    'Alabama',
    'Montana',
    'Alaska',
    'Nebraska',
    'Arizona',
    'Nevada',
    'Arkansas',
    'New Hampshire',
    'California',
    'New Jersey',
    'Colorado',
    'New Mexico',
    'Connecticut',
    'New York',
    'Delaware',
    'North Carolina',
    'Florida',
    'North Dakota',
    'Georgia',
    'Ohio',
    'Hawaii',
    'Oklahoma',
    'Idaho',
    'Oregon',
    'Illinois',
    'Pennsylvania',
    'Indiana',
    'Rhode Island',
    'Iowa',
    'South Carolina',
    'Kansas',
    'South Dakota',
    'Kentucky',
    'Tennessee',
    'Louisiana',
    'Texas',
    'Maine',
    'Utah',
    'Maryland',
    'Vermont',
    'Massachusetts',
    'Virginia',
    'Michigan',
    'Washington',
    'Minnesota',
    'West Virginia',
    'Mississippi',
    'Wisconsin',
    'Missouri',
    'Wyoming'
    ]

We'll turn these names into a set of dictionaries and add a few additional mappings that occur in the affiliation strings.

In [8]:
country_match = {cc:cc for cc in country_names}
country_match.update({cc.split(',')[0]:cc for cc in country_names if ',' in cc})
# Additional mappings
country_match.update({'czech republic':'czechia',
                      'españa':'spain',
                      'england':'united kingdom'})
countrycodes_match = {'usa':'united states',
                      'uk': 'united kingdom'}
state_match = {sn.lower():'united states' for sn in statenames}
statecodes_match = {sc.lower():'united states' for sc in statecodes}


To extract a country from an affiliation string, we'll first search to see if any of the country names match any of the whole words in the affiliation string. If so, we terminate the search. Then we do the same for US state names. Next we search for words that match some commonly abbreviated country names (USA and UK) and terminate the search if one is found. Finally, we search for any word that matches a state abbreviation and assign that affilation to the united states.

In [9]:
def simplematch(xstr, country_dict, state_dict, cc_dict, sc_dict):
    matches = []
    word_pat = re.compile(r'\b[a-z]+\b')
    xstr = xstr.replace('.','').lower()
    for yw in country_dict.keys():
        if len(re.findall(r'\b%s\b'%yw, xstr)) > 0:
            matches.append(country_dict[yw])
    if len(matches) > 0:
        matches = np.unique(matches)        
        return matches
    
    for yw in state_dict.keys():
        if len(re.findall(r'\b%s\b'%yw, xstr)) > 0:
            matches.append(state_dict[yw])
    if len(matches) > 0:
        matches = np.unique(matches)
        return matches
    
    for ww in re.findall(word_pat, xstr):
        if ww in cc_dict.keys():
            matches.append(cc_dict[ww])
    if len(matches) > 0:
        matches = np.unique(matches)
        return matches
    
    for ww in re.findall(word_pat, xstr):
        if ww in sc_dict.keys():
            matches.append(sc_dict[ww])
    if len(matches) > 0:
        matches = np.unique(matches)
        return matches
    return matches

We run our affiliation search for both first authors and last authors.

In [10]:
pmdf.loc[pd.notnull(pmdf.FirstAuth_aff) ,'FirstAuth_cl'] = pmdf.loc[pd.notnull(pmdf.FirstAuth_aff) ,'FirstAuth_aff'].apply(lambda x: simplematch(x, country_match, state_match, statecodes_match, countrycodes_match))
pmdf.loc[pd.notnull(pmdf.LastAuth_aff) ,'LastAuth_cl'] = pmdf.loc[pd.notnull(pmdf.LastAuth_aff) ,'LastAuth_aff'].apply(lambda x: simplematch(x, country_match, state_match, statecodes_match, countrycodes_match))


Next as a quality control step, we determine the number of times that a first or last author's country list has either no entries or more than one entry.

In [11]:
cl_list = []
for idx,cl in pmdf.loc[:,'FirstAuth_cl'].iteritems():
    if isinstance(cl,np.ndarray):
        cl_list.append(len(cl)!= 1)
    else:
        cl_list.append(False)
        
for idx,cl in pmdf.loc[:,'LastAuth_cl'].iteritems():
    if isinstance(cl,np.ndarray):
        cl_list[idx] = cl_list[idx] | (len(cl)!= 1)
    else:
        cl_list[idx] = cl_list[idx] | False

print("Number of times the first or last author has an affiliation entry, but either none or more than one country were found: %d " %np.sum(cl_list))
pmdf.loc[cl_list, :]


Number of times the first or last author has an affiliation entry, but either none or more than one country were found: 244 


,ArticleDay,ArticleMonth,ArticleTitle,ArticleYear,FirstAuth_aff,Language,LastAuth_aff,PMID,PublicationTypeList,not_review,FirstAuth_cl,LastAuth_cl
82,30,07,Neuroinflammation in Neurodegenerative Diseases: Current Multi-modal Imaging Studies and Future Opportunities for Hybrid PET/MRI.,2018.0,"NAPLab, IRCCS SDN Istituto di Ricerca Diagnostica e Nucleare, Via E. Gianturco 113, 80143 Naples, Italy.",eng,"Department of Biomedicine and Prevention, Faculty of Medicine, University of Rome ""Tor Vergata"", Rome, Italy; Athinoula A. Martinos Center for Biomedical Imaging, Massachusetts General Hospital and Harvard Medical School, Boston, United States. Electronic address: toschi@med.uniroma2.it.",30071279,"[Journal Article, Review]",False,[italy],"[italy, united states]"
106,26,07,The Role of Neuroimaging in the Diagnosis and Treatment of Depressive Disorder: A Recent Review.,2018.0,"Department of Nuclear Medicine, Xuanwu Hospital, Capital Medical University, Beijing. China.",eng,"Department of Radiology, China-Japan Friendship Hospital, Beijing. China.",30051778,[Journal Article],True,[china],"[china, japan]"
116,21,07,Hemispheric specialization of the basal ganglia during vocal emotion decoding: Evidence from asymmetric Parkinson's disease and,2018.0,"Neuroscience of Emotion and Affective Dynamics' laboratory, Department of Psychology and Educational Sciences, and Swiss Center for Affective Sciences, University of Geneva, Switzerland.",eng,"Neuroscience of Emotion and Affective Dynamics' laboratory, Department of Psychology and Educational Sciences, and Swiss Center for Affective Sciences, University of Geneva, Switzerland; Behavior and Basal Ganglia' research unit, University of Rennes 1, Rennes University Hospital, France; Neuropsychology Unit, Department of Neurology, University Hospitals of Geneva, Switzerland. Electronic address: julie.peron@unige.ch.",30040955,[Journal Article],True,[switzerland],"[france, switzerland]"
241,23,05,Spoken words are processed during dexmedetomidine-induced unresponsiveness.,2018.0,"Department of Psychology and Speech-Language Pathology, and Turku Brain and Mind Center, University of Turku, Turku, Finland; Department of Perioperative Services, Intensive Care and Pain Medicine, Turku University Hospital, Turku, Finland. Electronic address: roosa.kallionpaa@utu.fi.",eng,"Department of Psychology and Speech-Language Pathology, and Turku Brain and Mind Center, University of Turku, Turku, Finland; Department of Perioperative Services, Intensive Care and Pain Medicine, Turku University Hospital, Turku, Finland; Department of Cognitive Neuroscience and Philosophy, School of Bioscience, University of Skövde, Skövde, Sweden.",29935582,[Journal Article],True,[finland],"[finland, sweden]"
242,10,05,Dreaming and awareness during dexmedetomidine- and propofol-induced unresponsiveness.,2018.0,"Turku PET Centre, University of Turku and Turku University Hospital, Turku, Finland. Electronic address: liemra@utu.fi.",eng,"Department of Psychology and Speech-Language Pathology, and Turku Brain and Mind Center, University of Turku, Turku, Finland; Department of Perioperative Services, Intensive Care and Pain Medicine, Turku University Hospital, Turku, Finland; Department of Cognitive Neuroscience and Philosophy, School of Bioscience, University of Skövde, Sweden.",29935581,[Journal Article],True,[finland],"[finland, sweden]"
308,07,06,Dual-Modal NIR-Fluorophore Conjugated Magnetic Nanoparticle for Imaging Amyloid-β Species In Vivo.,2018.0,"Department Key Laboratory for Green Organic Synthesis and Application of Hunan Province, Key Laboratory of Environmentally Friendly Chemistry, Application of Ministry of Education, College of Chemistry, Xiangtan University, Xiangtan, 411105, China.",eng,"Department of Chemistry, Hong Kong Baptist University, Kowloon Tong, Hong Kong, SAR China.",29882247,[Journal Article],True,[china],"[china, hong kong]"
318,02,06,"Construction, expression, and characterization of AG1",2018.0,"Colleg

Combine the first author affiliations and last author affiliations. We'll call any publication that has the US listed in the first or last author's affiliations a US publication and anything else that has affiliation information, but doesn't list the United States will be called a non-US publication.

In [12]:
def combine_cls(row):
    try:
        return list(set(row.FirstAuth_cl).union(set(row.LastAuth_cl)))
    except TypeError:
        try:
            return list(set(row.FirstAuth_cl))
        except TypeError:
            try:
                return list(set(row.LastAuth_cl))
            except TypeError:
                return np.nan
pmdf["auth_cl"] = pmdf.apply(combine_cls, axis=1)
assert(pd.isnull(pmdf.auth_cl).sum() == (pd.isnull(pmdf.LastAuth_aff) & pd.isnull(pmdf.FirstAuth_aff)).sum())

pmdf.loc[pd.notnull(pmdf.auth_cl) ,'Auth_country'] = pmdf.loc[pd.notnull(pmdf.auth_cl) ,'auth_cl'].str[-1]
pmdf.loc[pd.notnull(pmdf.auth_cl) ,'Auth_USA'] = pmdf.loc[pd.notnull(pmdf.auth_cl) ,'auth_cl'].apply(lambda x: 'united states' in x)


Now that we have affiliations for everything, we can determine the number of US and number of non-US brain PET articles published per year, as well as the ratio of non-US to US pet articles per year.

In [13]:
world_pet_articles = pmdf.query('not_review & ArticleYear >=1990').groupby('ArticleYear').Auth_USA.count()
usa_pet_articles = pmdf.query('not_review & ArticleYear >=1990').groupby('ArticleYear').Auth_USA.sum()
non_usa_pet_articles = world_pet_articles - usa_pet_articles
nonus_to_us_ratio = non_usa_pet_articles / usa_pet_articles
nonus_to_us_ratio

ArticleYear
1990.0    1.637931
1991.0    1.670732
1992.0    1.772152
1993.0    1.785714
1994.0    2.122449
1995.0    1.816000
1996.0    1.670968
1997.0    1.789189
1998.0    2.071856
1999.0    2.087912
2000.0    2.202247
2001.0    2.470968
2002.0    2.377907
2003.0    2.405063
2004.0    2.215470
2005.0    2.302083
2006.0    2.111702
2007.0    2.062802
2008.0    2.242424
2009.0    2.538860
2010.0    2.711340
2011.0    2.511521
2012.0    2.520661
2013.0    2.027950
2014.0    2.622517
2015.0    2.845118
2016.0    2.493865
2017.0    2.796923
2018.0    2.240816
Name: Auth_USA, dtype: float64

## Combine clinical trials data and pubmed data
Now that we have the ratio of US to non-US studies from pubmed, we can combine the clinical trials and pubmed data to get an estimate of the number of PET studies per year. First we'll get a count of the total number of subjects in  completed studies each year, along with the number of studies and median number of subjects in each of those studies. Then we'll calculate the number of project non-us studies as the number of US studies from clinical trials multiplied by the ratio of non-US studies to US studies. Then we multiply this number by the median number of subjects per study, to get the projected number of subjects in Non-US Studies. We add this projection to the number of US subjects from clinicaltrials.gov to get the projected total brain PET studies per year. Finally, we multiply this by the approximate size of a single subject's worth of PET data on OpenNeuro.org (300 MB or 0.0003 TB) to get an estimate of the amount of data for brain PET studies generated per year.

In [14]:
ct_years = ct_df.query('Status == "Completed" | Status == "Withdrawn"').groupby(['complete_year'])['Enrollment'].agg(['sum','count', 'median']).reset_index()
ctpm_df = ct_years.merge(pd.DataFrame(nonus_to_us_ratio).reset_index(), how='left', left_on='complete_year',right_on='ArticleYear')
ctpm_df = ctpm_df.drop(columns='ArticleYear')
ctpm_df = ctpm_df.rename(columns={'sum':'total_us_subs','count':'n_us_studies','median':'median_us_subs', 'Auth_USA':'non_us_to_us_ratio'})
ctpm_df['projected_non_us_studies'] = ctpm_df.non_us_to_us_ratio * ctpm_df.n_us_studies
ctpm_df['projected_non_us_subs'] = ctpm_df.median_us_subs * ctpm_df['projected_non_us_studies']
ctpm_df['projected_total_subs'] = ctpm_df.projected_non_us_subs + ctpm_df.total_us_subs
ctpm_df['total_data'] = ctpm_df['projected_total_subs'] * 0.0003

Getting the amount of data collected in the last five complete years (2013-2017) is simple. We estimate the rate of data generation as the average yearly rate over those years, allowing us to estimate how much brain PET data might be available at the end of 2019.

In [15]:
data_last_five = ctpm_df.query('complete_year >= 2013 & complete_year <= 2017').total_data.sum()
data_per_year = ctpm_df.query('complete_year >= 2013 & complete_year <= 2017').total_data.mean()
data_2020 = data_last_five + (data_per_year * 3)

In [16]:
print('Data collected from 2013 to 2017 (last whole year): %0.2f TB' % data_last_five)
print('At a yearly rate of %0.2f TB, we expect %0.2f TB that could be shared by the end of 2020.' % (data_per_year, data_2020))

Data collected from 2013 to 2017 (last whole year): 7.13 TB
At a yearly rate of 1.43 TB, we expect 11.41 TB that could be shared by the end of 2020.
